In [13]:
import yelpapi
import json
import requests
from  pprint import  pprint
import pandas as pd
import numpy as np
import random

counties=pd.read_csv("acs2015_county_data.csv")
counties["county_string"] = counties["County"] + " County, " + counties["State"]
listofcounties = counties["county_string"].tolist()

In [14]:
zipcodes=pd.read_csv("zip_codes_states.csv")
zipcodes["county_string"] = zipcodes["county"] + " County, " + zipcodes["state"]
listofzipcodes=zipcodes["county_string"].unique().tolist()

In [51]:
#county list and zip list have discrepancy. need to pick counties that have entries in both lists

fullcountylist = list(set(listofcounties) & set(listofzipcodes))
sample_list = random.sample(fullcountylist,5)

In [52]:
#merged_df = zipcodes.merge(counties, left_on="county_string", right_on="county_string", how="left")
#maybe this wan't necessary but it's here if we need it
res_in_county = pd.DataFrame(sample_list)
res_in_county.columns = ["county_string"]
res_in_county["restaurant count"]=0
res_in_county["reviews"]=0
res_in_county["avg score"]=0
res_in_county["num of Italian"]=0
res_in_county["num of Mexican"]=0
res_in_county["num of Chinese"]=0
res_in_county["num of Indian"]=0
res_in_county



,county_string,restaurant count,reviews,avg score,num of Italian,num of Mexican,num of Chinese,num of Indian
0,"Steuben County, New York",0,0,0,0,0,0,0
1,"Garrett County, Maryland",0,0,0,0,0,0,0
2,"Marion County, Missouri",0,0,0,0,0,0,0
3,"Calaveras County, California",0,0,0,0,0,0,0
4,"Kennebec County, Maine",0,0,0,0,0,0,0


In [53]:
res_df = pd.DataFrame(columns=["name","zipcode","county_string","rating","review_count","category"])
res_df

,name,zipcode,county_string,rating,review_count,category


In [54]:
r = 0
for c in sample_list:
    county_string = str(c)
    for term in ["Italian","Mexican","Chinese","Indian"]:
        yelpkey="IXRN_1GI8f8J9Kj5O5GciqeyFSWR5RMh41_rADGy1KrtQ5ZABGGZEkWwot5mOka0NlzlNxXPmzYXLWk88oOV0N_YxD1MSW4kQRfJnFrwfHLtKrr2HcsjLo6uWBzAWnYx"
        response = requests.get(url="https://api.yelp.com/v3/businesses/search", 
                                params= {"location": county_string, 
                                         "term": term, "limit":"50"}, 
                                headers={'Authorization': 'bearer %s' % yelpkey})
        responsejson = response.json()
        
        try:
            count=0
            
            for x in range(0,49):
                for y in range(0,len(responsejson["businesses"][x]["categories"])):
                    if ((responsejson["businesses"][x]["categories"][y]["title"]==term) and 
                        (responsejson["businesses"][x]["review_count"] > 0)):
                        
                        zc = int(responsejson["businesses"][x]["location"]["zip_code"])
                        if (zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0]== county_string):
                            
                            print("\n",responsejson["businesses"][x]["name"],
                                  responsejson["businesses"][x]["location"]["zip_code"],
                                  zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0],
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  responsejson["businesses"][x]["categories"][y]["title"])
                            
                            res_df.loc[r]=[responsejson["businesses"][x]["name"],
                                  zc,
                                  county_string,
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  term]
                            count += 1
                            r += 1
        except:
            print("")
    
        print("End of list. Found", count, term, "restaurants in", county_string)
  



 Union Block Italian Bistro 14840 Steuben County, New York 4.5 116 Italian

 Marino's Restaurant 14843 Steuben County, New York 3.5 21 Italian

 Italian Villa Restaurant 14843 Steuben County, New York 3.5 10 Italian

 Sorge's Restaurant 14830 Steuben County, New York 3.5 78 Italian

 Aniello's Pizzeria 14830 Steuben County, New York 4.0 139 Italian

 Pizza Hut 14810 Steuben County, New York 3.5 2 Italian

 Angry Oven Pizzeria 14810 Steuben County, New York 3.0 21 Italian

 Waterfront Restaurant 14840 Steuben County, New York 3.5 35 Italian

 Giovanni's Pizzeria Restaurant 14843 Steuben County, New York 4.0 8 Italian

 Sumthin' Else 14830 Steuben County, New York 5.0 3 Italian

 Little Italy Of Hornell 14843 Steuben County, New York 4.5 2 Italian
End of list. Found 11 Italian restaurants in Steuben County, New York

 Fiesta Brava Mexican Restaurant 14830 Steuben County, New York 3.5 76 Mexican

 Little Boomers' Burrito Bar 14830 Steuben County, New York 4.0 39 Mexican

 Taco Bell 14810

In [57]:
res_df

,name,zipcode,county_string,rating,review_count,category
0,Union Block Italian Bistro,14840,"Steuben County, New York",4.5,116,Italian
1,Marino's Restaurant,14843,"Steuben County, New York",3.5,21,Italian
2,Italian Villa Restaurant,14843,"Steuben County, New York",3.5,10,Italian
3,Sorge's Restaurant,14830,"Steuben County, New York",3.5,78,Italian
4,Aniello's Pizzeria,14830,"Steuben County, New York",4.0,139,Italian
5,Pizza Hut,14810,"Steuben County, New York",3.5,2,Italian
6,Angry Oven Pizzeria,14810,"Steuben County, New York",3.0,21,Italian
7,Waterfront Restaurant,14840,"Steuben County, New York",3.5,35,Italian
8,Giovanni's Pizzeria Restaurant,14843,"Steuben County, New York",4.0,8,Italian
9,Sumthin' Else,14830,"Steuben County, New York",5.0,3,Italian


In [58]:
responsejson["businesses"]

[{'categories': [{'alias': 'indpak', 'title': 'Indian'}],
  'coordinates': {'latitude': 44.2475061250712,
   'longitude': -69.7659405693412},
  'display_phone': '(207) 203-0388',
  'distance': 18234.996396833827,
  'id': 'mughal-palace-chelsea',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/gQAK_6pPUXbkLoOq2wwGnQ/o.jpg',
  'is_closed': False,
  'location': {'address1': '730 River Rd',
   'address2': '',
   'address3': None,
   'city': 'Chelsea',
   'country': 'US',
   'display_address': ['730 River Rd', 'Chelsea, ME 04330'],
   'state': 'ME',
   'zip_code': '04330'},
  'name': 'Mughal Palace',
  'phone': '+12072030388',
  'rating': 4.5,
  'review_count': 21,
  'transactions': [],
  'url': 'https://www.yelp.com/biz/mughal-palace-chelsea?adjust_creative=G_DkR7WiUJ0P-HF5MN1vmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=G_DkR7WiUJ0P-HF5MN1vmg'},
 {'categories': [{'alias': 'indpak', 'title': 'Indian'}],
  'coordinates': {'latitude': 44.5497892901445,
   